<a href="https://colab.research.google.com/github/Ryan-RE-Wang/NTHU_ML-HW1/blob/main/ML_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import csv

df = pd.read_csv("data_1008.csv")

to_date = pd.to_datetime(df.pop("dateRep"), format='%d/%m/%Y')
df.index = to_date

countries = []
for i in range(len(df)):
  name = df['countriesAndTerritories'][i]
  if (countries.count(name) == 0):
    countries.append(name)

In [ ]:
# for creating csv file and TA testing

def create_sequence(X, X_log):
  offset = 7
  x_train = X_log[offset:]
  x_test = X_log[:offset]
  y_train = X['cases'][:len(X)-7]
  y_test = X['cases'][:offset]

  return x_train, y_train, x_test, y_test

In [ ]:
for training

def create_sequence(X, X_log):
  offset = 7
  x_train = X_log[offset+7:]
  x_test = X_log[offset:offset+7]
  y_train = X['cases'][offset:len(X)-7]
  y_test = X['cases'][:offset]

  return x_train, y_train, x_test, y_test


In [ ]:
def preprocess(X):

  X = X.fillna(0)

  X['day-1'] = X['cases'].shift(-1)
  
  X = X.iloc[:40]

  return X


In [ ]:
def log_transform(X_train):
  X_log = pd.DataFrame()

  X_log['cases'] = np.log10(X_train['cases'] + 1 - X_train['cases'].min())
  X_log['deaths'] = np.log10(X_train['deaths'] + 1 - X_train['deaths'].min())
  X_log['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'] = np.log10(X_train['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'] + 1 - X_train['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'].min())

  return X_log

In [ ]:
def model(X_train, y_train, X_test, y_test):

  regr = linear_model.LinearRegression()

  regr.fit(X_train, y_train)

  y_pred = regr.predict(X_test)

  y_pred_ = np.round(np.array(y_pred))
  Truth = np.array(y_test)

  to_replace = 0 if (np.median(y_pred_) < 0) else np.median(y_pred_)
  Predict = np.where(y_pred_ <= 0, to_replace, y_pred_)

  mape = []

  for i in range(len(Truth)):
    if (Truth[i] > 0):
      mape.append(np.abs(Truth[i]-Predict[i]) / np.abs(Truth[i]))
    else:
      mape.append(0)

  return Predict, np.mean(mape) * 100

In [ ]:
def test(Country):
  da = df.loc[(df['countriesAndTerritories'] == Country)]
  da = da.drop(['countriesAndTerritories', 'countryterritoryCode', 'geoId', 'continentExp', 'popData2019', 'year', 'month', 'day'], axis=1)
  
  da_log = log_transform(da)

  da_log_ = preprocess(da_log)

  da_ = preprocess(da)  

  X_train, y_train, X_test, y_test = create_sequence(da_, da_log_)

  Prediction, mape = model(X_train, y_train, X_test, y_test)

  return Prediction[::-1], mape


In [ ]:
# for write whole 210 countries to csv file

row_country = [' ']
row_0 = ['10/9']
row_1 = ['10/10']
row_2 = ['10/11']
row_3 = ['10/12']
row_4 = ['10/13']
row_5 = ['10/14']
row_6 = ['10/15']

for i in countries:
  row_country.append(i)
  forcast, mape_score = test(i)

  row_0.append(forcast[0])
  row_1.append(forcast[1])
  row_2.append(forcast[2])
  row_3.append(forcast[3])
  row_4.append(forcast[4])
  row_5.append(forcast[5])
  row_6.append(forcast[6])

with open('covid-19_prediction.csv', 'a', encoding="ISO-8859-1", newline='') as file:
    wr = csv.writer(file)

    wr.writerow(row_country)
    wr.writerow(row_0)
    wr.writerow(row_1)
    wr.writerow(row_2)
    wr.writerow(row_3)
    wr.writerow(row_4)
    wr.writerow(row_5)
    wr.writerow(row_6)

    file.close()

In [ ]:
#Testing only one country

country = 'Afghanistan'

forcast, mape_score = test(country)

print(forcast)

[ 47.  54. 102.  69.  70.  58.  48.]
